<a href="https://colab.research.google.com/github/MatthewSchofield25/Weather-Emergency-Application/blob/main/CS4485_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''

#from vaderSentiment.vaderSentiment
# JUST NOTES NOT CODE

import nltk
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import numpy as np

sentence = "I LOVE waking up in the middle of night. Yes. Totally."
sid_obj = SentimentIntensityAnalyzer()

    # polarity_scores method of SentimentIntensityAnalyzer object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
sentiment_dict = sid_obj.polarity_scores(sentence)
print(sentiment_dict['compound'])
print("Sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
print("Sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
print("Sentence was rated as ", sentiment_dict['pos']*100, "% Positive")

'''

0.824
Sentence was rated as  0.0 % Negative
Sentence was rated as  51.2 % Neutral
Sentence was rated as  48.8 % Positive


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Main Steps:
# Data Preprocessing: Tokenize data and remove punctuation. Don't forget to .lower
# NLP: Process Data; Detect frequency of words, n-grams (BOW), TF-IDF, possibly Glove Vectors.
# Model: We can use different models and test their accuracies. LTSM possibly


import pandas as pd
import numpy as np

import spacy
from geopy.geocoders import Nominatim

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Used for ML/Deep Learning Algorithms
from tensorflow.keras.layers import Embedding,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional,GRU,MaxPooling1D,Conv1D
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.preprocessing.text import Tokenizer

# Sentiment analysis
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score

# Labels on the data
#id : A unique identifier for each tweet.
#keyword : A particular keyword from the tweet (may be blank).
#location: The location the tweet was sent from (may be blank).
#text : The text of the tweet.
#target : This denotes whether a tweet is about a real disaster (1) or not (0).

common_words = ['via','like','build','get','would','one','two','feel','lol','fuck','take','way','may','first','latest'
                'want','make','back','see','know','let','look','come','got','still','say','think','great','pleas','amp']

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

ps = PorterStemmer()
lm = WordNetLemmatizer()

X = train.drop(columns=["target"],axis=1)
y = train["target"]

# Load the SpaCy NLP model
nlp = spacy.load('en_core_web_sm')

# Initialize the GeoPy geocoder
geolocator = Nominatim(user_agent='project_app')

def text_cleaning(data):
    return ' '.join(i for i in data.split() if i not in common_words)

def preprocess_data(data):
    '''
    Input: Data to be cleaned.
    Output: Cleaned Data.

    '''
    review =re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ',data) #removal of url
    review =re.sub(r'<.*?>',' ',review) #removal of html tags
    review = re.sub("["
                           u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+",' ',review)
    review = re.sub('[^a-zA-Z]',' ',review) # filtering out miscellaneous text.
    review = review.lower() # Lowering all the words in text
    review = review.split() # split into a list of words
    review = [lm.lemmatize(words) for words in review if words not in stopwords.words('english')] # Turn words into their stems/roots
    review = [i for i in review if len(i)>2] # Removal of words with length<2
    review = ' '.join(review) # Put back to single string with a space separator
    return review

def sentiment_ana(data):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(data)
    return sentiment_dict['compound']

def sentiment_ana_label(data):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(data)
    compound_score = sentiment_dict['compound']
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Define a function to extract location names from a text using SpaCy NER
def extract_locations(text):
    doc = nlp(text)
    #print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
    return [ent.text for ent in doc.ents if ent.label_ in ['LOC', 'GPE']]

# Define a function to map location names to countries using GeoPy geocoding
def map_locations_to_countries(locations):
    countries = set()
    for location in locations:
        try:
            location = geolocator.geocode(location, addressdetails=True, exactly_one=True, timeout=10)
            country = location.raw['address']['country']
            countries.add(country)
        except:
            pass
    return countries


def top_ngrams(data,n,grams):

    if grams == 1:
        count_vec = CountVectorizer(ngram_range=(1,1)).fit(data)
        bow = count_vec.transform(data)
        add_words = bow.sum(axis=0)
        word_freq = [(word, add_words[0, idx]) for word, idx in count_vec.vocabulary_.items()]
        word_freq = sorted(word_freq, key = lambda x: x[1], reverse=True)
    elif grams == 2:
        count_vec = CountVectorizer(ngram_range=(2,2)).fit(data)
        bow = count_vec.transform(data)
        add_words = bow.sum(axis=0)
        word_freq = [(word,add_words[0,idx]) for word,idx in count_vec.vocabulary_.items()]
        word_freq = sorted(word_freq, key = lambda x: x[1], reverse=True)
    elif grams == 3:
        count_vec = CountVectorizer(ngram_range=(3,3)).fit(data)
        bow = count_vec.transform(data)
        add_words = bow.sum(axis=0)
        word_freq = [(word,add_words[0,idx]) for word,idx in count_vec.vocabulary_.items()]
        word_freq = sorted(word_freq, key = lambda x: x[1], reverse=True)

    return word_freq[:n]

train["Cleaned_text"] = train["text"].apply(preprocess_data)
test["Cleaned_text"] = test["text"].apply(preprocess_data)

# Find common words and get rid of words that are unneeded
train["Cleaned_text"] = train["Cleaned_text"].apply(text_cleaning)
test["Cleaned_text"] = test["Cleaned_text"].apply(text_cleaning)

train["Sentiment"] = train["text"].apply(sentiment_ana)
test["Sentiment"] = test["text"].apply(sentiment_ana)

train["Sentiment_Label"] = train["text"].apply(sentiment_ana_label)
test["Sentiment_Label"] = test["text"].apply(sentiment_ana_label)

train["Location_Test"] = train["text"].apply(extract_locations)
#train["Country_Test"] = train["Location_Test"].apply(map_locations_to_countries)



common_words_uni = top_ngrams(train["Cleaned_text"],20,1)
common_words_bi = top_ngrams(train["Cleaned_text"],20,2)
common_words_tri = top_ngrams(train["Cleaned_text"],20,3)

print(common_words_uni)
print(common_words_bi)
print(common_words_tri)


#train.head(50)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


[('fire', 356), ('new', 228), ('news', 213), ('people', 201), ('time', 181), ('year', 178), ('video', 175), ('disaster', 162), ('emergency', 159), ('body', 155), ('day', 151), ('home', 144), ('police', 143), ('building', 141), ('life', 132), ('family', 132), ('storm', 128), ('crash', 125), ('california', 121), ('burning', 121)]
[('suicide bomber', 60), ('burning building', 59), ('body bag', 51), ('youtube video', 43), ('liked youtube', 42), ('northern california', 41), ('cross body', 40), ('oil spill', 39), ('suicide bombing', 36), ('california wildfire', 35), ('year old', 35), ('mass murder', 33), ('heat wave', 31), ('full read', 31), ('natural disaster', 31), ('mass murderer', 31), ('forest fire', 30), ('prebreak best', 30), ('bomber detonated', 30), ('home razed', 29)]
[('liked youtube video', 42), ('suicide bomber detonated', 30), ('northern california wildfire', 29), ('latest home razed', 28), ('home razed northern', 28), ('pkk suicide bomber', 28), ('bomber detonated bomb', 28), 

In [20]:
train.head(50)

,id,keyword,location,text,target,Cleaned_text,Sentiment,Sentiment_Label,Location_Test
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake allah forgive,0.2732,positive,[]
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near ronge sask canada,-0.3400,negative,[Canada]
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...,-0.2960,negative,[]
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...,0.0000,neutral,[California]
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,sent photo ruby alaska smoke wildfire pours sc...,0.0000,neutral,"[Ruby, Alaska]"
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,rockyfire update california hwy closed directi...,-0.3400,negative,[Lake County]
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,flood disaster heavy rain cause flash flooding...,0.0000,neutral,"[Manitou, Colorado Springs]"
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,top hill fire wood,-0.1531,negative,[]
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,emergency evacuation happening building across...,-0.3818,negative,[]
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,afraid tornado coming area,0.0000,neutral,[]


In [6]:
'''
def top_ngrams(data,n,grams):

    if grams == 1:
        count_vec = CountVectorizer(ngram_range=(1,1)).fit(data)
        bow = count_vec.transform(data)
        add_words = bow.sum(axis=0)
        word_freq = [(word, add_words[0, idx]) for word, idx in count_vec.vocabulary_.items()]
        word_freq = sorted(word_freq, key = lambda x: x[1], reverse=True)
    elif grams == 2:
        count_vec = CountVectorizer(ngram_range=(2,2)).fit(data)
        bow = count_vec.transform(data)
        add_words = bow.sum(axis=0)
        word_freq = [(word,add_words[0,idx]) for word,idx in count_vec.vocabulary_.items()]
        word_freq = sorted(word_freq, key = lambda x: x[1], reverse=True)
    elif grams == 3:
        count_vec = CountVectorizer(ngram_range=(3,3)).fit(data)
        bow = count_vec.transform(data)
        add_words = bow.sum(axis=0)
        word_freq = [(word,add_words[0,idx]) for word,idx in count_vec.vocabulary_.items()]
        word_freq = sorted(word_freq, key = lambda x: x[1], reverse=True)

    return word_freq[:n]

common_words_uni = top_ngrams(train["Cleaned_text"],20,1)
common_words_bi = top_ngrams(train["Cleaned_text"],20,2)
common_words_tri = top_ngrams(train["Cleaned_text"],20,3)

print(common_words_uni)
print(common_words_bi)
print(common_words_tri)
'''


[('fire', 356), ('new', 228), ('news', 213), ('people', 201), ('time', 181), ('year', 178), ('video', 175), ('disaster', 162), ('emergency', 159), ('body', 155), ('day', 151), ('home', 144), ('police', 143), ('building', 141), ('life', 132), ('family', 132), ('storm', 128), ('crash', 125), ('california', 121), ('burning', 121)]
[('suicide bomber', 60), ('burning building', 59), ('body bag', 51), ('youtube video', 43), ('liked youtube', 42), ('northern california', 41), ('cross body', 40), ('oil spill', 39), ('suicide bombing', 36), ('california wildfire', 35), ('year old', 35), ('mass murder', 33), ('heat wave', 31), ('full read', 31), ('natural disaster', 31), ('mass murderer', 31), ('forest fire', 30), ('prebreak best', 30), ('bomber detonated', 30), ('home razed', 29)]
[('liked youtube video', 42), ('suicide bomber detonated', 30), ('northern california wildfire', 29), ('latest home razed', 28), ('home razed northern', 28), ('pkk suicide bomber', 28), ('bomber detonated bomb', 28), 

In [12]:
'''
nltk.download('punkt_tab')
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers

# Function to preprocess and tokenize the text for Word2Vec
def tokenize_data(data):
    # Tokenize each cleaned text entry
    return [word_tokenize(review) for review in data]

# Tokenize the cleaned text in train and test datasets
train_tokens = tokenize_data(train["Cleaned_text"])
test_tokens = tokenize_data(test["Cleaned_text"])

# Train a Word2Vec model on the tokenized training data
word2vec_model = Word2Vec(sentences=train_tokens, vector_size=100, window=5, min_count=2, workers=4)

# Save the model for later use
word2vec_model.save("word2vec_model.model")


# Function to get the average word vector for a sentence
def get_sentence_embedding(sentence, model):
    word_vectors = [model.wv[word] for word in sentence if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # Return a zero vector if no words in the model
    return np.mean(word_vectors, axis=0)

# Generate sentence embeddings for the train and test sets
train_embeddings = [get_sentence_embedding(review, word2vec_model) for review in train_tokens]
test_embeddings = [get_sentence_embedding(review, word2vec_model) for review in test_tokens]


# Pad the sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in train_embeddings])  # Max length in the training set


train_embeddings_padded = pad_sequences(train_embeddings, maxlen=max_sequence_length, padding='post', dtype='float32')
test_embeddings_padded = pad_sequences(test_embeddings, maxlen=max_sequence_length, padding='post', dtype='float32')


embedding_feature_vector = 200 # Since we used glove vector embedding of dim 200.
model = Sequential()
model.add(LSTM(128, input_shape=(max_sequence_length, word2vec_model.vector_size), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification (for example)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])
print(model.summary())

# Get predictions on the test data
#predictions_prob = model.predict(test_embeddings_padded)  # Probability values between 0 and 1

# Convert probabilities to class labels (0 or 1)
#predictions = (predictions_prob > 0.5).astype(int)

# Show the first few predictions
#print(predictions[:10])  # Display the first 10 predictions

# If you want to evaluate the accuracy (if you have the true labels in test):
# If you have the true labels for the test set, you can compare the predictions with them:
# test_labels = ...  # Ensure you have the true labels for the test set
# accuracy = accuracy_score(test_labels, predictions)
# print(f'Accuracy on Test Data: {accuracy}')
'''


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                        │ (None, 128)                 │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 125,569 (490.50 KB)

 Trainable params: 125,569 (490.50 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
'''

import spacy
from geopy.geocoders import Nominatim

# Load the SpaCy NLP model
nlp = spacy.load('en_core_web_sm')

# Initialize the GeoPy geocoder
geolocator = Nominatim(user_agent='project_app')

# Define a function to extract location names from a text using SpaCy NER
def extract_locations(text):
    doc = nlp(text)
    print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
    return [ent.text for ent in doc.ents if ent.label_ in ['LOC', 'GPE']]

# Define a function to map location names to countries using GeoPy geocoding
def map_locations_to_countries(locations):
    countries = set()
    for location in locations:
        try:
            location = geolocator.geocode(location, addressdetails=True, exactly_one=True)
            country = location.raw['address']['country']
            countries.add(country)
        except:
            pass
    return countries

# Example text to infer country from
text = "I am traveling to Paris next month."

# Extract location names from the text
locations = extract_locations(text)

# Map location names to countries
countries = map_locations_to_countries(locations)

# Print the inferred countries
print(countries)

'''

Entities: [('Paris', 'GPE'), ('next month', 'DATE')]
{'France'}


In [ ]:
#traint = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ', traint)
#testt = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ', testt)

#train["new_text"] = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ', train["text"].apply)
#test["new_text"] = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ', test["text"].apply)

#newtrain =re.sub(r'<.*?>',' ',newtrain) #removal of html tags
#newtest =re.sub(r'<.*?>',' ',newtest)

#newtrain = re.sub(r'[^a-zA-Z]',' ',newtrain) #removal of punctuation
#newtest = re.sub(r'[^a-zA-Z]',' ',newtest)

#newtrain = re.sub("["
#                           u"\U0001F600-\U0001F64F"  # removal of emoticons
#                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                           u"\U00002702-\U000027B0"
#                           u"\U000024C2-\U0001F251"
#                           "]+",' ',newtrain)
#newtest = re.sub("["
#                           u"\U0001F600-\U0001F64F"  # removal of emoticons
#                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                           u"\U00002702-\U000027B0"
#                           u"\U000024C2-\U0001F251"
#                           "]+",' ',newtest)

#newtrain = newtrain.lower()
#newtest = newtest.lower()

#newtrain.head()

#newtest.head()

#token = Tokenizer()

In [13]:
'''

nltk.download('punkt_tab')
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

# Fit the tokenizer on the training and test data
tokenizer.fit_on_texts(train_tokens + test_tokens)

# Convert the tokens to sequences (list of indices)
train_sequences = tokenizer.texts_to_sequences(train_tokens)
test_sequences = tokenizer.texts_to_sequences(test_tokens)

# Set the max sequence length (you can modify this based on your analysis of sequence lengths)
max_timesteps = 100  # Adjust based on your dataset

# Pad the sequences
train_padded = pad_sequences(train_sequences, maxlen=max_timesteps, padding='post', truncating='post', dtype='int32')
test_padded = pad_sequences(test_sequences, maxlen=max_timesteps, padding='post', truncating='post', dtype='int32')

# Check the shapes of the padded sequences
print("Train Padded Shape:", train_padded.shape)
print("Test Padded Shape:", test_padded.shape)




# Train a Word2Vec model on the tokenized training data
word2vec_model = Word2Vec(sentences=train_tokens, vector_size=100, window=5, min_count=2, workers=4)

# Save the model for later use
word2vec_model.save("word2vec_model.model")


# Function to get embeddings for the padded sequences
def get_word_embeddings_for_sequences(sequences, word2vec_model, embedding_dim):
    embeddings = []
    for sequence in sequences:
        sequence_embeddings = []
        for word_index in sequence:
            # Convert word_index to integer to avoid the numpy.float32 issue
            word_index = int(word_index)

            # Skip padding (0) tokens
            if word_index != 0:
                # Convert word index to the corresponding word in the vocabulary
                # Use index_to_key instead of index2word
                if word_index < len(word2vec_model.wv.index_to_key):
                    word = word2vec_model.wv.index_to_key[word_index]
                    if word in word2vec_model.wv:
                        word_embedding = word2vec_model.wv[word]
                    else:
                        word_embedding = np.zeros(embedding_dim)  # Use zero vector if the word is not in vocabulary
                else:
                    word_embedding = np.zeros(embedding_dim)  # Use zero vector if index is out of vocabulary range
            else:
                word_embedding = np.zeros(embedding_dim)  # Padding is represented as a zero vector

            sequence_embeddings.append(word_embedding)
        embeddings.append(sequence_embeddings)

    return np.array(embeddings)

# Now, convert the padded sequences into word embeddings
embedding_dim = 100  # As per your Word2Vec model
train_embeddings = get_word_embeddings_for_sequences(train_padded, word2vec_model, embedding_dim)
test_embeddings = get_word_embeddings_for_sequences(test_padded, word2vec_model, embedding_dim)

# Check the shapes of the embeddings
print("Shape of train embeddings:", train_embeddings.shape)
print("Shape of test embeddings:", test_embeddings.shape)



# Generate sentence embeddings for the train and test sets
#train_embeddings = [get_sentence_embedding(review, word2vec_model) for review in train_tokens]
#test_embeddings = [get_sentence_embedding(review, word2vec_model) for review in test_tokens]


# Pad the sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in train_embeddings])  # Max length in the training set

train_embeddings_array = np.array(train_embeddings)
test_embeddings_array = np.array(test_embeddings)

# Now reshape the arrays
#train_embeddings_reshaped = train_embeddings_array.reshape((train_embeddings_array.shape[0], 1, 100))  # (samples, timesteps, features)
#test_embeddings_reshaped = test_embeddings_array.reshape((test_embeddings_array.shape[0], 1, 100))



# Now, let's reshape the padded sequences to match the expected LSTM input format
embedding_dim = 100  # Word2Vec embedding dimension

# Now, let's map the padded sequences to Word2Vec embeddings
train_embeddings = get_word_embeddings_for_sequences(train_padded, word2vec_model, embedding_dim)
test_embeddings = get_word_embeddings_for_sequences(test_padded, word2vec_model, embedding_dim)

# Reshape the data for LSTM: (samples, timesteps, features)
# train_embeddings and test_embeddings should already be of shape (samples, timesteps, embedding_dim)
print("Train Embeddings Shape:", train_embeddings.shape)
print("Test Embeddings Shape:", test_embeddings.shape)

# Reshaping the data for LSTM (samples, timesteps, features)
#train_padded_reshaped = train_padded.reshape((train_padded.shape[0], train_padded.shape[1], 100))  # (samples, timesteps, features)
#test_padded_reshaped = test_padded.reshape((test_padded.shape[0], test_padded.shape[1], 100))  # (samples, timesteps, features)

# Now you can pass this reshaped data into the LSTM model
#print("Shape of reshaped train data:", train_padded_reshaped.shape)
#print("Shape of reshaped test data:", test_padded_reshaped.shape)


# Update the LSTM layer's input shape to match the correct dimensions
#model = Sequential()
#model.add(LSTM(128, input_shape=(max_sequence_length, word2vec_model.vector_size), return_sequences=False))
#model.add(Dropout(0.2))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))  # Binary classification (for example)


model = Sequential()
model.add(LSTM(128, input_shape=(train_embeddings.shape[1], embedding_dim), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # For binary classification
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])


# Compile the model
#model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])

print(model.summary())

# Pad the sequences to ensure uniform length
max_sequence_length = max([len(seq) for seq in train_embeddings])  # Max length in the training set


# Fit the model
#model.fit(train_embeddings_reshaped, y_final, epochs=10, batch_size=32, validation_split=0.2)
model.fit(train_embeddings, y_final, epochs=5, batch_size=64)

y_pred = model.predict(test_embeddings)
y_pred = (y_pred > 0.5).astype(int)

# Get predictions on the test data
#predictions_prob = model.predict(test_embeddings_reshaped)


# Convert probabilities to class labels (0 or 1)
#predictions = (predictions_prob > 0.5).astype(int)

# Show the first few predictions
print(y_pred[:10])
'''

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Train Padded Shape: (7613, 100)
Test Padded Shape: (3263, 100)
Shape of train embeddings: (7613, 100, 100)
Shape of test embeddings: (3263, 100, 100)
Train Embeddings Shape: (7613, 100, 100)
Test Embeddings Shape: (3263, 100, 100)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 128)                 │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 125,569 (490.50 KB)

 Trainable params: 125,569 (490.50 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 23s 171ms/step - accuracy: 0.5726 - loss: 0.6899
Epoch 2/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 161ms/step - accuracy: 0.6218 - loss: 0.6476
Epoch 3/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 21s 167ms/step - accuracy: 0.6297 - loss: 0.6412
Epoch 4/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 21s 173ms/step - accuracy: 0.6299 - loss: 0.6434
Epoch 5/5
119/119 ━━━━━━━━━━━━━━━━━━━━ 40s 166ms/step - accuracy: 0.6213 - loss: 0.6468
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [19]:
'''
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam

# Define a dictionary of disaster types (example)
disaster_keywords = {
    'earthquake': ['earthquake', '#earthquake'],
    'flood': ['flood', '#flood'],
    'fire': ['fire', '#fire'],
    'storm': ['storm', '#storm'],
    # Add more disaster types as needed
}

def get_disaster_type(text):
    """ Return the type of disaster based on the tweet's content """
    for disaster, keywords in disaster_keywords.items():
        for keyword in keywords:
            if keyword.lower() in text.lower():
                return disaster
    return 'other'  # Default if no match

# Add a column for disaster type
train['disaster_type'] = train['text'].apply(get_disaster_type)

# Convert the disaster_type column to numeric labels for multi-class classification
disaster_types = train['disaster_type'].unique()
disaster_type_dict = {disaster: idx for idx, disaster in enumerate(disaster_types)}
train['disaster_type_label'] = train['disaster_type'].map(disaster_type_dict)


# Define input layer for the model
input_layer = Input(shape=(x_final.shape[1],))

# Embedding layer for word embeddings (we'll assume you have pre-trained word embeddings or are training your own)
embedding_layer = Embedding(input_dim=len(token.word_index) + 1, output_dim=100, input_length=x_final.shape[1])(input_layer)

# LSTM layer for sequence learning
lstm_layer = LSTM(128, return_sequences=True)(embedding_layer)
lstm_layer = Dropout(0.2)(lstm_layer)
lstm_layer = GlobalMaxPooling1D()(lstm_layer)

# Binary classification output (Disaster or not)
binary_output = Dense(1, activation='sigmoid', name='binary_output')(lstm_layer)

# Multi-class classification output (Type of disaster)
disaster_output = Dense(len(disaster_types), activation='softmax', name='disaster_output')(lstm_layer)

# Define the model
model = Model(inputs=input_layer, outputs=[binary_output, disaster_output])

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss={'binary_output': 'binary_crossentropy', 'disaster_output': 'categorical_crossentropy'},
              metrics={'binary_output': 'accuracy', 'disaster_output': 'accuracy'})

# Model summary
model.summary()

from tensorflow.keras.utils import to_categorical

# Convert the disaster labels to categorical for multi-class classification
y_disaster_type = to_categorical(y_final, num_classes=len(disaster_types))

# Train the model
model.fit(x_final, [y_final, y_disaster_type], epochs=5, batch_size=64, validation_split=0.2)

# Convert test labels to categorical for multi-class classification
y_test_disaster_type = to_categorical(y_test_final, num_classes=len(disaster_types))

# Evaluate the model
model.evaluate(x_test_final, [y_test_final, y_test_disaster_type])

# Predict on the test data
binary_pred, disaster_pred = model.predict(x_test_final)

# Convert binary predictions to labels
binary_pred_labels = (binary_pred > 0.5).astype(int)

# Convert disaster predictions to labels
disaster_pred_labels = disaster_pred.argmax(axis=-1)

# Example of predicted values
print(binary_pred_labels[:10])
print(disaster_pred_labels[:10])

'''


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 14486)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 14486, 100)     │      1,448,700 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ (None, 14486, 128)     │        117,248 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 14486, 128)     │              0 │ lstm_6[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_3    │ (None, 128)            │              0 │ dropout_6[0][0]        │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ binary_output (Dense)     │ (None, 1)              │            129 │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ disaster_output (Dense)   │ (None, 5)              │            645 │ global_max_pooling1d_… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,566,722 (5.98 MB)

 Trainable params: 1,566,722 (5.98 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 6715s 70s/step - binary_output_accuracy: 0.5709 - binary_output_loss: 0.6886 - disaster_output_accuracy: 0.4127 - disaster_output_loss: 1.3775 - loss: 2.0662 - val_binary_output_accuracy: 0.5345 - val_binary_output_loss: 0.6928 - val_disaster_output_accuracy: 0.5345 - val_disaster_output_loss: 0.7654 - val_loss: 1.4579
Epoch 2/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 6710s 70s/step - binary_output_accuracy: 0.5886 - binary_output_loss: 0.6776 - disaster_output_accuracy: 0.5886 - disaster_output_loss: 0.7081 - loss: 1.3858 - val_binary_output_accuracy: 0.5345 - val_binary_output_loss: 0.6909 - val_disaster_output_accuracy: 0.5345 - val_disaster_output_loss: 0.7266 - val_loss: 1.4173
Epoch 3/5
96/96 ━━━━━━━━━━━━━━━━━━━━ 6637s 69s/step - binary_output_accuracy: 0.5761 - binary_output_loss: 0.6819 - disaster_output_accuracy: 0.5761 - disaster_output_loss: 0.6957 - loss: 1.3776 - val_binary_output_accuracy: 0.5345 - val_binary_output_loss: 0.6917 - val_disaster_outp

NameError: name 'y_test_final' is not defined

In [3]:
# Define a dictionary of disaster types (example)
disaster_keywords = {
    'earthquake': ['earthquake', '#earthquake'],
    'flood': ['flood', '#flood'],
    'fire': ['fire', '#fire'],
    'storm': ['storm', '#storm'],
    'hurricane': ['hurricane', '#hurricane'],
    'tornado': ['tornado', '#tornado'],
    'tsunami': ['tsunami', '#tsunami'],
    'wildfire': ['wildfire', '#wildfire'],
    'drought': ['drought', '#drought'],
    'avalanche': ['avalanche', '#avalanche'],
    # Add more disaster types as needed
}

def get_disaster_type(text):
    """ Return the type of disaster based on the tweet's content """
    for disaster, keywords in disaster_keywords.items():
        for keyword in keywords:
            if keyword.lower() in text.lower():
                return disaster
    return 'other'  # Default if no match

# Add a column for disaster type
train['disaster_type'] = train['text'].apply(get_disaster_type)

# Convert the disaster_type column to numeric labels for multi-class classification
disaster_types = train['disaster_type'].unique()
disaster_type_dict = {disaster: idx for idx, disaster in enumerate(disaster_types)}
train['disaster_type_label'] = train['disaster_type'].map(disaster_type_dict)

train.head(100)


,id,keyword,location,text,target,Cleaned_text,Sentiment,Sentiment_Label,Location_Test,disaster_type,disaster_type_label
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake allah forgive,0.2732,positive,[],earthquake,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near ronge sask canada,-0.3400,negative,[Canada],fire,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident asked shelter place notified officer ...,-0.2960,negative,[],other,2
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...,0.0000,neutral,[California],fire,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,sent photo ruby alaska smoke wildfire pours sc...,0.0000,neutral,"[Ruby, Alaska]",fire,1
...,...,...,...,...,...,...,...,...,...,...,...
95,137,accident,Charlotte,9 Mile backup on I-77 South...accident blockin...,1,mile backup south accident blocking right lane...,-0.3818,negative,"[NC, NC, NC]",other,2
96,138,accident,"Baton Rouge, LA",Has an accident changed your life? We will hel...,0,accident changed life help determine option fi...,0.6705,positive,[],other,2
97,139,accident,"Hagerstown, MD",#BREAKING: there was a deadly motorcycle car a...,1,breaking deadly motorcycle car accident happen...,-0.4767,negative,[Hagerstown],other,2
98,141,accident,"Gloucestershire , UK",@flowri were you marinading it or was it an ac...,0,flowri marinading accident,-0.4767,negative,[@flowri],other,2


In [10]:
'''

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dropout, LSTM, Dense
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.feature_extraction.text import TfidfVectorizer


#X = train["Cleaned_text"]
#y = train["target"]

# Convert text data to TF-IDF representation
#vectorizer = TfidfVectorizer(max_features=5000)
#X_tfidf = vectorizer.fit_transform(X)

# Now split the data (80% train, 20% validation)
#X_train_tfidf, X_val_tfidf, y_train, y_val = train_test_split(
#    X_tfidf, y, test_size=0.2, random_state=42
#)

# Check the shapes to ensure they match
#print(f"X_train_tfidf shape: {X_train_tfidf.shape}")
#print(f"y_train shape: {y_train.shape}")
#print(f"X_val_tfidf shape: {X_val_tfidf.shape}")
#print(f"y_val shape: {y_val.shape}")


# Step 1: Convert text data into TF-IDF numerical representation
vectorizer = TfidfVectorizer(max_features=5000)  # Using top 5000 words as features
X_train_tfidf = vectorizer.fit_transform(train["Cleaned_text"])  # Fit on training data
X_test_tfidf = vectorizer.transform(test["Cleaned_text"])  # Transform test data

# Step 2: Set target values
y_train = train["target"]

# Step 3: Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

print(f"Length of X_train_tfidf: {X_train_tfidf.shape[0]}")
print(f"Length of y_train: {len(y_train)}")
print(f"Length of X_val: {X_val.shape[0]}")
print(f"Length of y_val: {len(y_val)}")

# Model definition
#embedding_feature_vector = 200  # Since we used Word2Vec vectors of dim 200.
#model = Sequential()
#model.add(Embedding(vocab_size, embedding_feature_vector, input_length=max_length, weights=[word_vector_matrix], trainable=False))
#model.add(Dropout(0.35))
#model.add(LSTM(200))
#model.add(Dropout(0.35))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))

# Compile the model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#print(model.summary())

# Callbacks
#n_epoch = 30
#early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min')

# Train the model
#history = model.fit(X_train, Y_train, validation_data=(x_valid, y_valid),
#                    callbacks=[reduce_lr, early_stop], epochs=n_epoch, batch_size=64)


# Step 4: Define the model architecture
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))  # Input layer with TF-IDF features
model.add(Dropout(0.35))  # Dropout layer for regularization
model.add(Dense(128, activation='relu'))  # Hidden layer
model.add(Dropout(0.35))  # Dropout layer for regularization
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Step 5: Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Step 6: Define callbacks
n_epoch = 30
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min')

# Step 7: Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    callbacks=[reduce_lr, early_stop], epochs=n_epoch, batch_size=64)


# Assuming the previous steps have been followed, like training the model and defining the feature vectors

# Get TF-IDF features for the test data (this is already done in your pipeline)
# predictions from the model
predictions = model.predict(X_val_tfidf)

# Convert probabilities to binary values (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Print the first 10 predictions
print("First 10 predictions on validation set (1 = disaster, 0 = not disaster):")
print(binary_predictions[:10])

# If you want to see the actual prediction probabilities (between 0 and 1)
print("First 10 raw prediction probabilities:")
print(predictions[:10])

accuracy = accuracy_score(y_val, binary_predictions)

# Output the overall accuracy
print(f"Validation accuracy: {accuracy * 100:.2f}%")

predictions = model.predict(X_test_tfidf)

# Convert probabilities to binary values (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

print("First 10 predictions on test set (1 = disaster, 0 = not disaster):")
print(binary_predictions[:10])

'''

Length of X_train_tfidf: 7613
Length of y_train: 6090
Length of X_val: 1523
Length of y_val: 1523


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                     │ (None, 512)                 │       2,560,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,630,337 (10.03 MB)

 Trainable params: 2,630,337 (10.03 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.6152 - loss: 0.6480 - val_accuracy: 0.7886 - val_loss: 0.4700 - learning_rate: 0.0010
Epoch 2/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.8789 - loss: 0.3162 - val_accuracy: 0.7827 - val_loss: 0.4806 - learning_rate: 0.0010
Epoch 3/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.9276 - loss: 0.1998 - val_accuracy: 0.7715 - val_loss: 0.5829 - learning_rate: 0.0010
Epoch 4/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.9574 - loss: 0.1223 - val_accuracy: 0.7702 - val_loss: 0.7132 - learning_rate: 0.0010
Epoch 5/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.9675 - loss: 0.0919 - val_accuracy: 0.7676 - val_loss: 0.8338 - learning_rate: 0.0010
Epoch 6/30
95/96 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9783 - loss: 0.0607
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.9782 - loss: 0.0610 - va

In [4]:
def encoding(train_data,test_data):
    tfidf = TfidfVectorizer(
          ngram_range=(1, 1), use_idf=True, smooth_idf=True, sublinear_tf=True
    )
    tf_df_train = tfidf.fit_transform(train_data).toarray()
    train_df = pd.DataFrame(tf_df_train,columns=tfidf.get_feature_names_out())
    tf_df_test = tfidf.transform(test_data).toarray()
    test_df = pd.DataFrame(tf_df_test,columns=tfidf.get_feature_names_out())

    return train_df,test_df

x_final,x_test_final = encoding(train["Cleaned_text"],test["Cleaned_text"])
y_final = np.array(y)

x_final.shape,y_final.shape,x_test_final.shape

# Dividing the data into training, validation and testing
from sklearn.model_selection import train_test_split
# for bow and tf-idf
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.1, random_state=42, stratify = y_final)
X_train, x_valid, Y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=42, stratify = y_train)
x_test_final = x_test_final

from keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers

embedding_feature_vector = 200 # Since we used glove vector embedding of dim 200.
#model = Sequential()
#model.add(Embedding(vocab_size,embedding_feature_vector,input_length=max_length,weights = [word_vector_matrix], trainable = False))
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))  # Input layer with TF-IDF features
model.add(Dropout(0.35))  # Dropout layer for regularization
model.add(Dense(128, activation='relu'))  # Hidden layer
model.add(Dropout(0.35))  # Dropout layer for regularization
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

from tensorflow.keras.callbacks import *
n_epoch = 30

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1,
                           mode='min', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5,
                              verbose=1, mode='min')

history = model.fit(X_train,Y_train,validation_data=(x_valid,y_valid),callbacks=[reduce_lr,early_stop],epochs=n_epoch,batch_size= 64)

predictions = model.predict(x_valid)

# Convert probabilities to binary values (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

# Print the first 10 predictions
print("First 10 predictions on validation set (1 = disaster, 0 = not disaster):")
print(binary_predictions[:10])

# If you want to see the actual prediction probabilities (between 0 and 1)
print("First 10 raw prediction probabilities:")
print(predictions[:10])

accuracy = accuracy_score(y_valid, binary_predictions)

# Output the overall accuracy
print(f"Validation accuracy: {accuracy * 100:.2f}%")

predictions = model.predict(x_test_final)

# Convert probabilities to binary values (0 or 1)
binary_predictions = (predictions > 0.5).astype(int)

print("First 10 predictions on test set (1 = disaster, 0 = not disaster):")
print(binary_predictions[:10])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │       7,417,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,487,169 (28.56 MB)

 Trainable params: 7,487,169 (28.56 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/30
97/97 ━━━━━━━━━━━━━━━━━━━━ 19s 157ms/step - accuracy: 0.5850 - loss: 0.6490 - val_accuracy: 0.8134 - val_loss: 0.4410 - learning_rate: 0.0010
Epoch 2/30
97/97 ━━━━━━━━━━━━━━━━━━━━ 20s 159ms/step - accuracy: 0.8945 - loss: 0.2827 - val_accuracy: 0.7959 - val_loss: 0.4829 - learning_rate: 0.0010
Epoch 3/30
97/97 ━━━━━━━━━━━━━━━━━━━━ 20s 151ms/step - accuracy: 0.9574 - loss: 0.1266 - val_accuracy: 0.7872 - val_loss: 0.5902 - learning_rate: 0.0010
Epoch 4/30
97/97 ━━━━━━━━━━━━━━━━━━━━ 16s 161ms/step - accuracy: 0.9717 - loss: 0.0778 - val_accuracy: 0.7799 - val_loss: 0.6649 - learning_rate: 0.0010
Epoch 5/30
97/97 ━━━━━━━━━━━━━━━━━━━━ 25s 212ms/step - accuracy: 0.9785 - loss: 0.0525 - val_accuracy: 0.7930 - val_loss: 0.7258 - learning_rate: 0.0010
Epoch 6/30
97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.9829 - loss: 0.0393
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
97/97 ━━━━━━━━━━━━━━━━━━━━ 36s 163ms/step - accuracy: 0.9828 - loss